In [1]:
import pandas as pd
from datasets import load_dataset
from PIL import Image
import numpy as np
import time
import openai
import os

C:\Users\wreep\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\wreep\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
C:\Users\wreep\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\wreep\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core imp

In [ ]:

dataset = load_dataset('poloclub/diffusiondb', 'large_first_1k', trust_remote_code=True)
df=pd.DataFrame(dataset)
df.head()

README.md:   0%|          | 0.00/25.0k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/25.0k [00:00<?, ?B/s]

C:\Users\wreep\AppData\Roaming\Python\Python39\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\wreep\.cache\huggingface\hub\datasets--poloclub--diffusiondb. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
C:\Users\wreep\AppData\Roaming\Python\Python39\site-packages\huggingface_hub\file_download.py:142: User

diffusiondb.py:   0%|          | 0.00/15.7k [00:00<?, ?B/s]

diffusiondb.py:   0%|          | 0.00/15.7k [00:00<?, ?B/s]

part-000002.zip:   0%|          | 0.00/512M [00:00<?, ?B/s]

part-000002.zip:   0%|          | 0.00/512M [00:00<?, ?B/s]

In [ ]:

dataset = load_dataset('poloclub/diffusiondb', 'large_first_50k', trust_remote_code=True)




In [ ]:

batch_size = 10000
df_list = []  # List to store DataFrame batches

# Process dataset in batches
for i in range(0, len(dataset['train']), batch_size):
    batch = dataset['train'].select(range(i, min(i + batch_size, len(dataset['train']))))
    df_batch = batch.to_pandas()
    df_list.append(df_batch)  # Append batch to list

# Concatenate all batches into a final DataFrame
prompts = pd.concat(df_list, ignore_index=True)
secdeset = pd.read_csv('diffusion_db_unaltered.csv')
prompts = pd.concat([prompts, secdeset], ignore_index=True)

prompts.info()
# Display the first few rows
prompts.head()
prompts.tail()

In [ ]:
image_path = prompts.loc[0, 'image']['path']  # Extract the image path
image = Image.open(image_path)
image.show()
image_array = np.array(image)
print(image_array.shape)

In [ ]:
image_rgb = image.convert('RGB')
image_array = np.expand_dims(image_rgb, axis=0)


In [ ]:
print(image_array)

In [ ]:
openai.api_key = os.getenv("OPENAI_API_KEY")
import getpass
if openai.api_key is None:
    print(' ValueError("API key is missing. Please set the OPENAI_API_KEY environment variable.")')
    openai.api_key = getpass.getpass("Enter your API key: ")
# Initialize the OpenAI client
client = openai

In [ ]:
def classify_word(word):
    # Define a system message and user prompt to classify each word
    prompt = f"Classify the word '{word}' into one of the following categories: 'illegal', 'explicit', 'violent', 'innocent'. Provide just the category as the output. Ensure one of these categories is no matter what stated."

    # Call the OpenAI API with the prompt
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that classifies words."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=10
        )
        
        # Parse the response and extract the classification
        classification = response.choices[0].message.content.strip()
        if 'illegal' in classification:
            classification = 'illegal'
        elif 'explicit' in classification:
            classification = 'explicit'
        elif 'violent' in classification:
            classification = 'violent'
        elif 'innocent' in classification:
            classification = 'innocent'
        else:
            response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that classifies words."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=30
        )
        # Map the response to a unique value
        category_mapping = {
            "illegal": 1,
            "explicit": 2,
            "violent": 3,
            "innocent": 0
        }
        
        # Return the mapped value
        return category_mapping.get(classification, -1)  # Return -1 for unknown categories
    
    except Exception as e:
        print(f"Error classifying word '{word}': {e}")
        return -1

In [ ]:
import re

def numberify(word):
    # Define the user prompt
    prompt = f"HERE IS THE WORD: {word}"

    try:
        # First classification: 4-digit number
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": """Assign the word a number based on the following criteria.
    The number will be 4 digits. The first 4 digits determine how 'bad' the word may be.
    - Filler words = 1111
    - Non-explicit subjects = 2222
    - Non-explicit verbs = 3333
    - Explicit words increase progressively (e.g., 5555, 6666, 7777, 8888)
    - Use 9999 only for extreme cases.
    Respond with just the number!."""},
                {"role": "user", "content": prompt}
            ],
            max_tokens=10
        )
        classification = response.choices[0].message.content.strip()

        # Second classification: 2-digit number based on implication
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": """Assign the word a number from 00 to 99 based on severity.
    - Use a balanced scale with 99 reserved for extreme words.
    Respond with just the number!."""},
                {"role": "user", "content": prompt}
            ],
            max_tokens=10
        )
        secondary = response.choices[0].message.content.strip()

        # Third classification: Letter count in 2-digit format
        letter_count = len(re.findall(r'[a-zA-Z]', word))
        formatted_count = str(letter_count).zfill(2)  # Ensures 2-digit format

        # Combine all values into a single 8-digit number
        final_number = f"{classification}{formatted_count}{secondary}"

        return final_number, word  # Returns the combined number and word
    
    except Exception as e:
        print(f"Error classifying word '{word}': {e}")
        return -1, word  # Returns -1 on error



In [ ]:
df.info()

In [ ]:
"""
import re
from tqdm import tqdm

# Example DataFrame (replace this with your actual DataFrame)

word_classifications = {}
rows = []
words_set = set()
# Loop through the DataFrame and classify words across all rows
for idx, row in tqdm(prompts.iterrows(), total=len(prompts), desc="Processing prompts"):
    prompt = row['prompt']
    for word in re.findall(r'\b\w+\b', prompt):  # Split the prompt into individual words
        if word not in words_set:  # Avoid classifying the same word twice
            words_set.add(word)
            word_classifications[word] = {}
            classification_value = classify_word(word)
            numeric_value, interpreted_word=numberify(word)
            rows.append([word, classification_value, numeric_value, interpreted_word])
            print(word, classification_value, numeric_value, interpreted_word)
        # Sleep to avoid hitting API rate limits
        time.sleep(.1)  # Adjust the sleep time as necessary

# Convert the dictionary to a DataFrame for better visualization
word_classification_df = pd.DataFrame(rows, columns=["Word", "Classification", "Number", "NumericInterpretedWord"])
# Display the final DataFrame with words and their classifications
print(word_classification_df)
"""

In [ ]:
import re
import pandas as pd
from tqdm import tqdm
from collections import OrderedDict

def normalize_text(text):
    replacements = {
        '3': 'E', '5': 'S', '0': 'O', '1': 'I', '2': 'Z'
    }
    for key, value in replacements.items():
        text = text.replace(key, value)
    return text

# Maintain word order while ensuring uniqueness
ordered_words = OrderedDict()
prompts['prompt'] = prompts['prompt'].fillna("").astype(str)

# Extract words in order from prompts
for prompt in tqdm(prompts['prompt'], desc="Extracting unique words"):
    prompt = normalize_text(prompt)
    words = re.findall(r'\b\w+\b', prompt)
    for word in words:
        if word not in ordered_words:  # Preserve only first occurrence
            ordered_words[word] = None  

# Classify words in preserved order
rows = []
for word in tqdm(ordered_words.keys(), desc="Classifying words"):
    classification_value = classify_word(word)
    numeric_value, interpreted_word = numberify(word)
    rows.append([word, classification_value, numeric_value, interpreted_word])

df = pd.DataFrame(rows, columns=["Word", "Classification", "Number", "NumericInterpretedWord"])

print(df)

In [ ]:
valid_rows = []
for word in tqdm(ordered_words.keys(), desc="Validating words"):
    existing_row = next((row for row in rows if row[0] == word), None)

    if existing_row:
        classification_value = existing_row[1]  
        numeric_value = existing_row[2]     
        interpreted_word = existing_row[3]

        # Recompute only if Classification or Number is -1
        if classification_value == -1:
            classification_value = classify_word(word)
        if numeric_value == -1:
            numeric_value, interpreted_word = numberify(word)

    else:  # If word is missing from rows, compute everything
        classification_value = classify_word(word)
        numeric_value, interpreted_word = numberify(word)

    valid_rows.append([word, classification_value, numeric_value, interpreted_word])


# Update dataframe with validated values
df = pd.DataFrame(valid_rows, columns=["Word", "Classification", "Number", "NumericInterpretedWord"])

In [ ]:
print(df['Classification'].unique())

In [ ]:
df = pd.DataFrame(rows, columns=["Word", "Classification", "Number", "NumericInterpretedWord"])
df.info

In [ ]:
def classify_wordSPECIALIZED(word):
    # Define a system message and user prompt to classify each word
    prompt = f"Classify the word '{word}' into one of the following categories: 'illegal', 'explicit', 'violent', 'innocent'. Provide just the category as the output. Ensure one of these categories is no matter what stated."

    # Call the OpenAI API with the prompt
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that classifies words."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=50
        )
        
        # Parse the response and extract the classification
        classification = response.choices[0].message.content.strip()
        print(classification)
        if 'illegal' in classification:
            classification = 'illegal'
        elif 'explicit' in classification:
            classification = 'explicit'
        elif 'violent' in classification:
            classification = 'violent'
        elif 'innocent' in classification:
            classification = 'innocent'
        else:
            response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that classifies words."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=30
        )
            classification = response.choices[0].message.content.strip()
            print(classification)
            if 'illegal' in classification:
                classification = 'illegal'
            elif 'explicit' in classification:
                classification = 'explicit'
            elif 'violent' in classification:
                classification = 'violent'
            elif 'innocent' in classification:
                classification = 'innocent'
        # Map the response to a unique value
        category_mapping = {
            "illegal": 1,
            "explicit": 2,
            "violent": 3,
            "innocent": 0
        }
        
        # Return the mapped value
        return category_mapping.get(classification, -1)  # Return -1 for unknown categories
    
    except Exception as e:
        print(f"Error classifying word '{word}': {e}")
        return -1

In [ ]:
for index, row in tqdm(df.iterrows(), total=len(df), desc="Validating words"):
    classification_value = row["Classification"]
    numeric_value = row["Number"]

    # Recompute only if Classification or Number is -1
    if classification_value == -1:
        df.at[index, "Classification"] = classify_wordSPECIALIZED(row["Word"])
    if numeric_value == -1:
        df.at[index, "Number"], df.at[index, "NumericInterpretedWord"] = numberify(row["Word"])

In [ ]:
print(df.loc[df['Classification'] == -1])

In [ ]:
df.loc[df['Classification'] == -1, 'Classification'] = 0

In [ ]:
len(df)

In [ ]:
count_class_1 = len(df.loc[df['Classification'] == 1])
count_class_2 = len(df.loc[df['Classification'] == 2])
count_class_3 = len(df.loc[df['Classification'] == 3])

# Print the results
print(f"Count of Classification 1: {count_class_1}")
print(f"Count of Classification 2: {count_class_2}")
print(f"Count of Classification 3: {count_class_3}")

In [ ]:
print(df.loc[df['Classification'] == 2])

In [ ]:
df.to_csv('WordList.csv', index=False)


In [ ]:
"""
Conditions: combination of 2 or more illicit classifications 1+2, 2+3, 1+3
Repetition: 3 or more repitions of the same illicit classification
average of numbers: if the average number is over 50000000
High number: If there is any instance of a number over 60000000
"""

In [ ]:
SafePrompt='Generate an image of a child looking to the starry night'
NSFWPrompt='Generate an image of a busty looking female with big boobs'

In [ ]:
prompts.head()

In [ ]:
prompts['Banned'] = (prompts['prompt_nsfw'] > 0.5).astype(int)

In [ ]:
prompts.head()

In [ ]:
print(prompts['allowed'].unique())

In [ ]:
prompts.info()

In [ ]:
import re
from tqdm import tqdm
df_dict = df.set_index('Word')[['Classification', 'Number']].to_dict(orient='index')

for idx, row in tqdm(prompts.iterrows(), total=len(prompts), desc="Processing prompts"):
    Classprompt = row['prompt']
    Numprompt = row['prompt']

    Classprompt = re.sub(r'[^a-zA-Z\s]', '', Classprompt)  
    Numprompt = re.sub(r'[^a-zA-Z\s]', '', Numprompt) 
    for word in re.findall(r'\b\w+\b', Classprompt):  
        if word in df_dict:
            classification_value = df_dict[word]['Classification']
            numeric_value = df_dict[word]['Number']
        else:
            classification_value = classify_word(word)
            numeric_value, _ = numberify(word)
        Classprompt = re.sub(r'\b' + re.escape(word) + r'\b', str(classification_value), Classprompt)
        Numprompt = re.sub(r'\b' + re.escape(word) + r'\b', str(numeric_value), Numprompt)

    prompts.at[idx, 'Classprompt'] = str(Classprompt) 
    prompts.at[idx, 'Numprompt'] = str(Numprompt) 


In [ ]:
prompts.head(867760)

In [ ]:
#pre filtering
prompts['Class 3 appearance count'] = prompts['Classprompt'].str.count('3')
prompts['Class 2 appearance count'] = prompts['Classprompt'].str.count('2')
prompts['Class 1 appearance count'] = prompts['Classprompt'].str.count('1')

prompts['Class Sum count'] = (
    prompts['Class 3 appearance count'] +
    prompts['Class 2 appearance count'] +
    prompts['Class 1 appearance count']
)

prompts.loc[prompts['Class 3 appearance count'] > 4, 'Guess allowed'] = 0
prompts.loc[prompts['Class 2 appearance count'] > 4, 'Guess allowed'] = 0
prompts.loc[prompts['Class 1 appearance count'] > 4, 'Guess allowed'] = 0
prompts.loc[prompts['Class Sum count'] > 4, 'Guess allowed'] = 0

prompts['Numbers'] = prompts['Numprompt'].fillna("").astype(str).str.findall(r'\d+').apply(lambda x: list(map(int, x)) if x else [])

prompts['Average Number'] = prompts['Numbers'].apply(lambda x: sum(x)/len(x) if x else 0)
prompts.loc[prompts['Average Number'] > 50000000, 'Guess allowed'] = 0
prompts.loc[prompts['Average Number'] <= 50000000, 'Guess allowed'] = 1

prompts.loc[prompts['Numbers'].apply(lambda x: any(num > 60000000 for num in x)), 'Guess allowed'] = 0

"""
Conditions: combination of 2 or more illicit classifications 1+2, 2+3, 1+3
Repetition: 3 or more repitions of the same illicit classification
average of numbers: if the average number is over 50000000
High number: If there is any instance of a number over 60000000
"""
prompts.head()
#prompts.tail()

In [ ]:

pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_colwidth', None)
print(prompts.iloc[2])

In [ ]:
df_subset = prompts.head(867760)
if 'Guess allowed' in df_subset.columns and 'allowed' in df_subset.columns:
    true_positives = (df_subset['Guess allowed'] == 1) & (df_subset['allowed'] == 1)

    true_negatives = (df_subset['Guess allowed'] == 0) & (df_subset['allowed'] == 0)

    false_positives = (df_subset['Guess allowed'] == 1) & (df_subset['allowed'] == 0)

    false_negatives = (df_subset['Guess allowed'] == 0) & (df_subset['allowed'] == 1)

    tp_count = true_positives.sum()
    tn_count = true_negatives.sum()
    fp_count = false_positives.sum()
    fn_count = false_negatives.sum()

    print(f"True Positives (TP): {tp_count}")
    print(f"True Negatives (TN): {tn_count}")
    print(f"False Positives (FP): {fp_count}")
    print(f"False Negatives (FN): {fn_count}")
    total = len(df_subset)

    tp_percentage = (tp_count / total) * 100
    tn_percentage = (tn_count / total) * 100
    fp_percentage = (fp_count / total) * 100
    fn_percentage = (fn_count / total) * 100

    print(f"True Positives Ratio: {tp_percentage:.2f}%")
    print(f"True Negatives Ratio: {tn_percentage:.2f}%")
    print(f"False Positives Ratio: {fp_percentage:.2f}%")
    print(f"False Negatives Ratio: {fn_percentage:.2f}%")

    accuracy = (tp_count + tn_count) / len(df_subset)
    print(f"Accuracy: {accuracy * 100:.2f}%")
else:
    print("The required columns are not present in the first 100 rows.")

In [ ]:
import re
from tqdm import tqdm

# Define your custom prompt here (replace this with your actual prompt)
custom_prompt = NSFWPrompt

# Initialize the word classifications (use your pre-built logic)
word_classifications = {}
rows = []

# Process the custom prompt
Classprompt = custom_prompt
Numprompt = custom_prompt

# Clean up non-alphabetical characters (remove special characters)
Classprompt = re.sub(r'[^a-zA-Z\s]', '', Classprompt)  
Numprompt = re.sub(r'[^a-zA-Z\s]', '', Numprompt)

# Process words and classify them
for word in re.findall(r'\b\w+\b', Classprompt):  # Loop through each word
    if word not in word_classifications:
        classification_value = classify_word(word)  # Classify word (you'll have to define this)
        numeric_value, interpreted_word = numberify(word)  # Interpret numeric value (define this too)
        word_classifications[word] = {'Classification': classification_value, 
                                       'Number': numeric_value, 'NumericInterpretedWord': interpreted_word}
    else:
        classification_value = word_classifications[word]['Classification']
        numeric_value = word_classifications[word]['Number']
    
    # Replace the word in the prompts with the classification and numeric value
    Classprompt = re.sub(r'\b' + re.escape(word) + r'\b', str(classification_value), Classprompt)
    Numprompt = re.sub(r'\b' + re.escape(word) + r'\b', str(numeric_value), Numprompt)

# Output the processed prompt with classifications and numeric values
print("Processed Classprompt:", Classprompt)
print("Processed Numprompt:", Numprompt)

# Pre-filtering based on the same logic as before
Class_3_count = Classprompt.count('3')
Class_2_count = Classprompt.count('2')
Class_1_count = Classprompt.count('1')

Class_Sum_count = Class_3_count + Class_2_count + Class_1_count

# Apply filtering conditions
Guess_allowed = 1  # Default assumption (if conditions are not violated)
if Class_3_count > 4 or Class_2_count > 4 or Class_1_count > 4 or Class_Sum_count > 4:
    Guess_allowed = 0

# Extract numbers from Numprompt
Numbers = list(map(int, re.findall(r'\d+', Numprompt)))
Average_Number = sum(Numbers)/len(Numbers) if Numbers else 0

if Average_Number > 50000000:
    Guess_allowed = 0
elif any(num > 60000000 for num in Numbers):
    Guess_allowed = 0

# Print the result for the custom prompt
print(f"Guess allowed: {Guess_allowed}")

# Assuming you have the actual label `allowed` for your custom prompt (manually define it here)
# For this example, let's assume the actual label is stored in `actual_allowed`:
actual_allowed = 0  # You would replace this with the actual expected result for your custom prompt

# Calculate the accuracy or compare the results
if Guess_allowed == actual_allowed:
    print("Prediction is correct.")
else:
    print("Prediction is incorrect.")


In [ ]:
import re
from tqdm import tqdm

# Define your custom prompt here (replace this with your actual prompt)
custom_prompt = SafePrompt

# Initialize the word classifications (use your pre-built logic)
word_classifications = {}
rows = []

# Process the custom prompt
Classprompt = custom_prompt
Numprompt = custom_prompt

# Clean up non-alphabetical characters (remove special characters)
Classprompt = re.sub(r'[^a-zA-Z\s]', '', Classprompt)  
Numprompt = re.sub(r'[^a-zA-Z\s]', '', Numprompt)

# Process words and classify them
for word in re.findall(r'\b\w+\b', Classprompt):  # Loop through each word
    if word not in word_classifications:
        classification_value = classify_word(word)  # Classify word (you'll have to define this)
        numeric_value, interpreted_word = numberify(word)  # Interpret numeric value (define this too)
        word_classifications[word] = {'Classification': classification_value, 
                                       'Number': numeric_value, 'NumericInterpretedWord': interpreted_word}
    else:
        classification_value = word_classifications[word]['Classification']
        numeric_value = word_classifications[word]['Number']
    
    # Replace the word in the prompts with the classification and numeric value
    Classprompt = re.sub(r'\b' + re.escape(word) + r'\b', str(classification_value), Classprompt)
    Numprompt = re.sub(r'\b' + re.escape(word) + r'\b', str(numeric_value), Numprompt)

# Output the processed prompt with classifications and numeric values
print("Processed Classprompt:", Classprompt)
print("Processed Numprompt:", Numprompt)

# Pre-filtering based on the same logic as before
Class_3_count = Classprompt.count('3')
Class_2_count = Classprompt.count('2')
Class_1_count = Classprompt.count('1')

Class_Sum_count = Class_3_count + Class_2_count + Class_1_count

# Apply filtering conditions
Guess_allowed = 1  # Default assumption (if conditions are not violated)
if Class_3_count > 4 or Class_2_count > 4 or Class_1_count > 4 or Class_Sum_count > 4:
    Guess_allowed = 0

# Extract numbers from Numprompt
Numbers = list(map(int, re.findall(r'\d+', Numprompt)))
Average_Number = sum(Numbers)/len(Numbers) if Numbers else 0

if Average_Number > 50000000:
    Guess_allowed = 0
elif any(num > 60000000 for num in Numbers):
    Guess_allowed = 0

# Print the result for the custom prompt
print(f"Guess allowed: {Guess_allowed}")

# Assuming you have the actual label `allowed` for your custom prompt (manually define it here)
# For this example, let's assume the actual label is stored in `actual_allowed`:
actual_allowed = 1  # You would replace this with the actual expected result for your custom prompt

# Calculate the accuracy or compare the results
if Guess_allowed == actual_allowed:
    print("Prediction is correct.")
else:
    print("Prediction is incorrect.")


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt


In [ ]:
X = df_subset['Numbers'].apply(lambda x: sum(x) if isinstance(x, list) else 0).values.reshape(-1, 1)  # If 'Numbers' is a list, sum them as a feature
y = df_subset['allowed']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
accuracies, precisions, recalls, f1_scores = [], [], [], []
tp_values, tn_values, fp_values, fn_values = [], [], [], []

# Loop over different values of k and compute the accuracy
for k in range(1, 13):  # Test values of k from 1 to 12
    print("="*50)
    print(f"Results for k={k}")
    print("="*50)
    
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    
    # Step 5: Make Predictions
    y_pred = knn.predict(X_test)
    
    # Step 6: Evaluate the Model
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Accuracy for k={k}: {accuracy * 100:.2f}%')
    
    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    print(f"\nConfusion Matrix for k={k}:")
    print(cm)
    
    # Extract the components of the confusion matrix
    tn, fp, fn, tp = cm.ravel()
    
    print(f"\nTrue Positives (TP): {tp}")
    print(f"True Negatives (TN): {tn}")
    print(f"False Positives (FP): {fp}")
    print(f"False Negatives (FN): {fn}")
    
    # Additional metrics (optional)
    precision = tp / (tp + fp) if tp + fp > 0 else 0
    recall = tp / (tp + fn) if tp + fn > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1_score)
    tp_values.append(tp)
    tn_values.append(tn)
    fp_values.append(fp)
    fn_values.append(fn)
    print(f"Precision for k={k}: {precision * 100:.2f}%")
    print(f"Recall for k={k}: {recall * 100:.2f}%")
    print(f"F1 Score for k={k}: {f1_score * 100:.2f}%")
    
    # Print the first 5 predicted values and their actual counterparts
    print("\nFirst 5 predictions vs actual values:")
    for i in range(5):
        print(f"Sample {i+1} - Predicted: {y_pred[i]}, Actual: {y_test.iloc[i]}")
    
    print("\n" + "="*50 + "\n")
k_values = list(range(1, 13))

fig, axes = plt.subplots(3, 2, figsize=(12, 12))  # 3 rows, 2 columns

# Define plot data
metrics = [
    (tp_values, "True Positives", "green", "o"),
    (tn_values, "True Negatives", "blue", "s"),
    (fp_values, "False Positives", "red", "^"),
    (fn_values, "False Negatives", "orange", "v"),
    (accuracies, "Accuracy", "purple", "D")
]

# Plot each metric
for i, (values, title, color, marker) in enumerate(metrics):
    row, col = divmod(i, 2)  # Calculate subplot position
    axes[row, col].plot(k_values, values, marker=marker, linestyle="-", color=color, label=title)
    axes[row, col].set_xlabel("k (Number of Neighbors)")
    axes[row, col].set_ylabel("Count" if title != "Accuracy" else "Score")
    axes[row, col].set_title(f"{title} Across Different k Values")
    axes[row, col].grid(True)
    axes[row, col].legend()

# Hide the empty subplot (bottom-right)
fig.delaxes(axes[2, 1])

# Adjust layout
plt.subplots_adjust(hspace=0.4)
plt.show()

In [ ]:
import re
import numpy as np
from tqdm import tqdm

# Assuming the model 'knn' is already trained

# Define your custom prompt here (replace this with your actual prompt)
custom_prompt = NSFWPrompt

# Initialize the word classifications (use your pre-built logic)
word_classifications = {}
rows = []

# Process the custom prompt
Classprompt = custom_prompt
Numprompt = custom_prompt

# Clean up non-alphabetical characters (remove special characters)
Classprompt = re.sub(r'[^a-zA-Z\s]', '', Classprompt)  
Numprompt = re.sub(r'[^a-zA-Z\s]', '', Numprompt)

# Process words and classify them
for word in re.findall(r'\b\w+\b', Classprompt):  # Loop through each word
    if word not in word_classifications:
        # Here, classify the word based on your own logic, e.g., classify_word, numberify
        classification_value = classify_word(word)  # Classify word (you'll have to define this)
        numeric_value, interpreted_word = numberify(word)  # Interpret numeric value (define this too)
        word_classifications[word] = {'Classification': classification_value, 
                                       'Number': numeric_value, 'NumericInterpretedWord': interpreted_word}
    else:
        classification_value = word_classifications[word]['Classification']
        numeric_value = word_classifications[word]['Number']
    
    # Replace the word in the prompts with the classification and numeric value
    Classprompt = re.sub(r'\b' + re.escape(word) + r'\b', str(classification_value), Classprompt)
    Numprompt = re.sub(r'\b' + re.escape(word) + r'\b', str(numeric_value), Numprompt)

# Output the processed prompt with classifications and numeric values
print("Processed Classprompt:", Classprompt)
print("Processed Numprompt:", Numprompt)

# Pre-filtering based on the same logic as before
Class_3_count = Classprompt.count('3')
Class_2_count = Classprompt.count('2')
Class_1_count = Classprompt.count('1')

Class_Sum_count = Class_3_count + Class_2_count + Class_1_count

# Apply filtering conditions
Guess_allowed = 1  # Default assumption (if conditions are not violated)
if Class_3_count > 4 or Class_2_count > 4 or Class_1_count > 4 or Class_Sum_count > 4:
    Guess_allowed = 0

# Extract numbers from Numprompt
Numbers = list(map(int, re.findall(r'\d+', Numprompt)))
Average_Number = sum(Numbers)/len(Numbers) if Numbers else 0

if Average_Number > 50000000:
    Guess_allowed = 0
elif any(num > 60000000 for num in Numbers):
    Guess_allowed = 0

# Print the result for the custom prompt
print(f"Guess allowed: {Guess_allowed}")

# Assuming you have the actual label `allowed` for your custom prompt (manually define it here)
# For this example, let's assume the actual label is stored in `actual_allowed`:
actual_allowed = 0  # You would replace this with the actual expected result for your custom prompt

# Calculate the accuracy or compare the results
if Guess_allowed == actual_allowed:
    print("Filter is correct.")
else:
    print("Filter is incorrect.")

# Preprocess the custom prompt (as you did with your training data)
def preprocess_custom_prompt(prompt):
    # Extract numbers from the prompt
    numbers = re.findall(r'\d+', prompt)
    
    # Convert the numbers to integers
    numbers = list(map(int, numbers))
    
    # Sum the numbers (or any other feature engineering you want to use)
    sum_of_numbers = sum(numbers) if numbers else 0
    
    # Return the feature as an array (reshape to match the model input)
    return np.array([[sum_of_numbers]])

# Preprocess the custom prompt for KNN input
X_custom = preprocess_custom_prompt(custom_prompt)

# Predict the class for the custom prompt using KNN
knn = KNeighborsClassifier(n_neighbors=2)
knn.fit(X_train, y_train)
custom_prediction = knn.predict(X_custom)
if custom_prediction == actual_allowed:
    print("KNN Prediction is correct.")
else:
    print("KNN Prediction is incorrect.")
# Output the KNN prediction result
print(f"KNN Prediction for the custom prompt: {custom_prediction[0]}")


In [ ]:
import re
import numpy as np
from tqdm import tqdm

# Assuming the model 'knn' is already trained

# Define your custom prompt here (replace this with your actual prompt)
custom_prompt = SafePrompt

# Initialize the word classifications (use your pre-built logic)
word_classifications = {}
rows = []

# Process the custom prompt
Classprompt = custom_prompt
Numprompt = custom_prompt

# Clean up non-alphabetical characters (remove special characters)
Classprompt = re.sub(r'[^a-zA-Z\s]', '', Classprompt)  
Numprompt = re.sub(r'[^a-zA-Z\s]', '', Numprompt)

# Process words and classify them
for word in re.findall(r'\b\w+\b', Classprompt):  # Loop through each word
    if word not in word_classifications:
        # Here, classify the word based on your own logic, e.g., classify_word, numberify
        classification_value = classify_word(word)  # Classify word (you'll have to define this)
        numeric_value, interpreted_word = numberify(word)  # Interpret numeric value (define this too)
        word_classifications[word] = {'Classification': classification_value, 
                                       'Number': numeric_value, 'NumericInterpretedWord': interpreted_word}
    else:
        classification_value = word_classifications[word]['Classification']
        numeric_value = word_classifications[word]['Number']
    
    # Replace the word in the prompts with the classification and numeric value
    Classprompt = re.sub(r'\b' + re.escape(word) + r'\b', str(classification_value), Classprompt)
    Numprompt = re.sub(r'\b' + re.escape(word) + r'\b', str(numeric_value), Numprompt)

# Output the processed prompt with classifications and numeric values
print("Processed Classprompt:", Classprompt)
print("Processed Numprompt:", Numprompt)

# Pre-filtering based on the same logic as before
Class_3_count = Classprompt.count('3')
Class_2_count = Classprompt.count('2')
Class_1_count = Classprompt.count('1')

Class_Sum_count = Class_3_count + Class_2_count + Class_1_count

# Apply filtering conditions
Guess_allowed = 1  # Default assumption (if conditions are not violated)
if Class_3_count > 4 or Class_2_count > 4 or Class_1_count > 4 or Class_Sum_count > 4:
    Guess_allowed = 0

# Extract numbers from Numprompt
Numbers = list(map(int, re.findall(r'\d+', Numprompt)))
Average_Number = sum(Numbers)/len(Numbers) if Numbers else 0

if Average_Number > 50000000:
    Guess_allowed = 0
elif any(num > 60000000 for num in Numbers):
    Guess_allowed = 0

# Print the result for the custom prompt
print(f"Guess allowed: {Guess_allowed}")

# Assuming you have the actual label `allowed` for your custom prompt (manually define it here)
# For this example, let's assume the actual label is stored in `actual_allowed`:
actual_allowed = 1  # You would replace this with the actual expected result for your custom prompt

# Calculate the accuracy or compare the results
if Guess_allowed == actual_allowed:
    print("Filter is correct.")
else:
    print("Filter is incorrect.")

# Preprocess the custom prompt (as you did with your training data)
def preprocess_custom_prompt(prompt):
    # Extract numbers from the prompt
    numbers = re.findall(r'\d+', prompt)
    
    # Convert the numbers to integers
    numbers = list(map(int, numbers))
    
    # Sum the numbers (or any other feature engineering you want to use)
    sum_of_numbers = sum(numbers) if numbers else 0
    
    # Return the feature as an array (reshape to match the model input)
    return np.array([[sum_of_numbers]])

# Preprocess the custom prompt for KNN input
X_custom = preprocess_custom_prompt(custom_prompt)

# Predict the class for the custom prompt using KNN
knn = KNeighborsClassifier(n_neighbors=2)
knn.fit(X_train, y_train)
custom_prediction = knn.predict(X_custom)
if custom_prediction == actual_allowed:
    print("KNN Prediction is correct.")
else:
    print("KNN Prediction is incorrect.")
# Output the KNN prediction result
print(f"KNN Prediction for the custom prompt: {custom_prediction[0]}")


In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE()
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [ ]:
X = df_subset['Numbers'].apply(lambda x: sum(x) if isinstance(x, list) else 0).values.reshape(-1, 1)
y = df_subset['allowed']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
# Initialize Naïve Bayes model
nb = GaussianNB()
nb.fit(X_train_resampled, y_train_resampled)

# Make Predictions
y_pred = nb.predict(X_test)

# Compute Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()

# Store metrics for graphing
accuracies = [accuracy_score(y_test, y_pred)]
tp_values = [tp]
tn_values = [tn]
fp_values = [fp]
fn_values = [fn]

# Print results
print("="*50)
print("Results for Naïve Bayes Classifier")
print("="*50)
print(f"\nConfusion Matrix:\n{cm}")
print(f"\nTrue Positives (TP): {tp}")
print(f"True Negatives (TN): {tn}")
print(f"False Positives (FP): {fp}")
print(f"False Negatives (FN): {fn}")

# Compute additional metrics
precision = tp / (tp + fp) if tp + fp > 0 else 0
recall = tp / (tp + fn) if tp + fn > 0 else 0
f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

print(f"\nAccuracy: {accuracies[0] * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1 Score: {f1_score * 100:.2f}%")

# Create bar plot for the confusion matrix components
fig, ax = plt.subplots(figsize=(10, 6))
metrics = ['True Positives', 'True Negatives', 'False Positives', 'False Negatives']
values = [tp, tn, fp, fn]
colors = ['green', 'blue', 'red', 'orange']

ax.bar(metrics, values, color=colors)
ax.set_ylabel("Count")
ax.set_title("Confusion Matrix Breakdown for Naïve Bayes")
ax.grid(axis='y', linestyle='--', alpha=0.7)

plt.show()

In [ ]:
X = df_subset['Numbers'].apply(lambda x: sum(x) if isinstance(x, list) else 0).values.reshape(-1, 1)  # If 'Numbers' is a list, sum them as a feature
y = df_subset['allowed']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)


accuracies, precisions, recalls, f1_scores = [], [], [], []
tp_values, tn_values, fp_values, fn_values = [], [], [], []

# Loop over different values of k and compute the accuracy
for k in range(1, 13):  # Test values of k from 1 to 12
    print("="*50)
    print(f"Results for k={k}")
    print("="*50)
    
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_resampled, y_train_resampled)
    
    # Step 5: Make Predictions
    y_pred = knn.predict(X_test)
    
    # Step 6: Evaluate the Model
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Accuracy for k={k}: {accuracy * 100:.2f}%')
    
    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    print(f"\nConfusion Matrix for k={k}:")
    print(cm)
    
    # Extract the components of the confusion matrix
    tn, fp, fn, tp = cm.ravel()
    
    print(f"\nTrue Positives (TP): {tp}")
    print(f"True Negatives (TN): {tn}")
    print(f"False Positives (FP): {fp}")
    print(f"False Negatives (FN): {fn}")
    
    # Additional metrics (optional)
    precision = tp / (tp + fp) if tp + fp > 0 else 0
    recall = tp / (tp + fn) if tp + fn > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1_score)
    tp_values.append(tp)
    tn_values.append(tn)
    fp_values.append(fp)
    fn_values.append(fn)
    print(f"Precision for k={k}: {precision * 100:.2f}%")
    print(f"Recall for k={k}: {recall * 100:.2f}%")
    print(f"F1 Score for k={k}: {f1_score * 100:.2f}%")
    
    # Print the first 5 predicted values and their actual counterparts
    print("\nFirst 5 predictions vs actual values:")
    for i in range(5):
        print(f"Sample {i+1} - Predicted: {y_pred[i]}, Actual: {y_test.iloc[i]}")
    
    print("\n" + "="*50 + "\n")
k_values = list(range(1, 13))

fig, axes = plt.subplots(3, 2, figsize=(12, 12))  # 3 rows, 2 columns

# Define plot data
metrics = [
    (tp_values, "True Positives", "green", "o"),
    (tn_values, "True Negatives", "blue", "s"),
    (fp_values, "False Positives", "red", "^"),
    (fn_values, "False Negatives", "orange", "v"),
    (accuracies, "Accuracy", "purple", "D")
]

# Plot each metric
for i, (values, title, color, marker) in enumerate(metrics):
    row, col = divmod(i, 2)  # Calculate subplot position
    axes[row, col].plot(k_values, values, marker=marker, linestyle="-", color=color, label=title)
    axes[row, col].set_xlabel("k (Number of Neighbors)")
    axes[row, col].set_ylabel("Count" if title != "Accuracy" else "Score")
    axes[row, col].set_title(f"{title} Across Different k Values")
    axes[row, col].grid(True)
    axes[row, col].legend()

# Hide the empty subplot (bottom-right)
fig.delaxes(axes[2, 1])

# Adjust layout
plt.subplots_adjust(hspace=0.4)
plt.show()